# Scanning Session Ingestion

## Setup

### Connect to the database

If you are don't have your login information, contact the administrator.

Using local config file (see [01_pipeline](./01_pipeline.ipynb)):

https://github.com/datajoint/workflow-calcium-imaging/blob/main/notebooks/03-process.ipynb
https://github.com/datajoint/workflow-calcium-imaging/blob/main/notebooks/04-automate-optional.ipynb

Check the Moser lab descriptions of useage as well.
https://github.com/kavli-ntnu/dj-docs/blob/master/notebooks/2022-11%20Imaging%20workshop/Working_with_Imaging_pipeline.ipynb

Check the youtube: https://www.youtube.com/watch?v=gFLn0GB1L30

In [ ]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
import datajoint as dj; dj.conn()

from adamacs.pipeline import subject, session, surgery, scan, event, trial, imaging
from adamacs import utility
from adamacs.ingest import behavior as ibe
import numpy as np
import scanreader
from tifffile.tifffile import matlabstr2py

### Creating a Parameter Set

Some parameters will be loaded from your `dj.config` file. For instance the fast disc to be used. This will also depend on the location from which you access datajoint. For example if you use our other analysis server, this would be a local temp file.

In [ ]:
dj.config['custom'].get('suite2p_fast_tmp')[0]

What exactly happens during processing dependso on the parameter set. Datajoint will read `fs`, `nchannels`and `nplanes` directly from the scaninfo. The values in here will be overwritten. This is an example of a parameter set and its insert:

In [ ]:
# Insert the param_set - Mini2p

# example for setting a link fro the classifier to be used for cell curation. Needs
classifier_folder = "/home/tobiasr/adamacs/user_data/s2p_classifiers/TR_HPC_bench2p_classifier.npy"

params_suite2p = {'look_one_level_down': False,
                  'fast_disk': dj.config['custom'].get('suite2p_fast_tmp')[0],
                  'delete_bin': True,
                  'mesoscan': False,
                  'h5py': [],
                  'h5py_key': 'data',
                  'save_path0': [],
                  'subfolders': [],
                  'nplanes': 1,
                  'nchannels': 1,
                  'functional_chan': 1,
                  'tau': 1.0,
                  'fs': 15.3845,
                  'force_sktiff': False,
                  'preclassify': 0.0,
                  'use_builtin_classifier': True,
                #   'classifier_path': classifier_folder,
                  'save_mat': True,
                  'combined': True,
                  'aspect': 1.0,
                  'do_bidiphase': True,
                  'bidiphase': 0.0,
                  'do_registration': True,
                  'two_step_registration': True,
                  'norm_frames': True,
                  'keep_movie_raw': False,
                  'nimg_init': 1000,
                  'batch_size': 8000,
                  'maxregshift': 0.1,
                  'align_by_chan': 1,
                  'reg_tif': True,
                  'reg_tif_chan2': False,
                  'subpixel': 10,
                  'smooth_sigma_time': 1, 
                  'smooth_sigma': 1.15,
                  'th_badframes': 1.0,
                  'pad_fft': False,
                  'nonrigid': True,
                  'block_size': [64, 64],
                  'snr_thresh': 1.2,
                  'maxregshiftNR': 5.0,
                  '1Preg': False,
                  'spatial_hp': 50.0,
                  'pre_smooth': 2.0,
                  'spatial_taper': 50.0,
                  'roidetect': True,
                  'sparse_mode': True,
                  'denoise': True,
                  'diameter': 6,
                  'spatial_scale': 0,
                  'connected': True,
                  'nbinned': 9000,
                  'max_iterations': 30,
                  'threshold_scaling': 1.0,
                  'max_overlap': 0.75,
                  'high_pass': 100.0,
                  'inner_neuropil_radius': 2,
                  'min_neuropil_pixels': 350,
                  'allow_overlap': False,
                  'chan2_thres': 0.65,
                  'baseline': 'maximin',
                  'win_baseline': 60.0,
                  'sig_baseline': 10.0,
                  'prctile_baseline': 8.0,
                  'neucoeff': 0.7,
                  'xrange': np.array([0, 0]),
                  'yrange': np.array([0, 0])}


In [ ]:
(imaging.ProcessingParamSet & "paramset_idx = 0").delete()
imaging.ProcessingParamSet.insert_new_params(
    processing_method="suite2p",
    concatenation_method="indiv",
    paramset_idx=0,
    params=params_suite2p,
    paramset_desc="TR: Mini2p (non-rigid, built-in classifier)",
)


In [ ]:
# Insert the param_set - Mini2p - rigid

# example for setting a link fro the classifier to be used for cell curation. Needs
classifier_folder = "/home/tobiasr/adamacs/user_data/s2p_classifiers/TR_HPC_bench2p_classifier.npy"

params_suite2p = {'look_one_level_down': False,
                  'fast_disk': dj.config['custom'].get('suite2p_fast_tmp')[0],
                  'delete_bin': True,
                  'mesoscan': False,
                  'h5py': [],
                  'h5py_key': 'data',
                  'save_path0': [],
                  'subfolders': [],
                  'nplanes': 1,
                  'nchannels': 1,
                  'functional_chan': 1,
                  'tau': 1.0,
                  'fs': 15.3845,
                  'force_sktiff': False,
                  'preclassify': 0.0,
                  'use_builtin_classifier': True,
                #   'classifier_path': classifier_folder,
                  'save_mat': True,
                  'combined': True,
                  'aspect': 1.0,
                  'do_bidiphase': True,
                  'bidiphase': 0.0,
                  'do_registration': True,
                  'two_step_registration': True,
                  'norm_frames': True,
                  'keep_movie_raw': False,
                  'nimg_init': 1000,
                  'batch_size': 8000,
                  'maxregshift': 0.1,
                  'align_by_chan': 1,
                  'reg_tif': False,
                  'reg_tif_chan2': False,
                  'subpixel': 10,
                  'smooth_sigma_time': 1, 
                  'smooth_sigma': 1.15,
                  'th_badframes': 1.0,
                  'pad_fft': False,
                  'nonrigid': False,
                  'block_size': [64, 64],
                  'snr_thresh': 1.2,
                  'maxregshiftNR': 5.0,
                  '1Preg': False,
                  'spatial_hp': 50.0,
                  'pre_smooth': 2.0,
                  'spatial_taper': 50.0,
                  'roidetect': True,
                  'sparse_mode': True,
                  'denoise': True,
                  'diameter': 6,
                  'spatial_scale': 0,
                  'connected': True,
                  'nbinned': 9000,
                  'max_iterations': 30,
                  'threshold_scaling': 1.0,
                  'max_overlap': 0.75,
                  'high_pass': 100.0,
                  'inner_neuropil_radius': 2,
                  'min_neuropil_pixels': 350,
                  'allow_overlap': False,
                  'chan2_thres': 0.65,
                  'baseline': 'maximin',
                  'win_baseline': 60.0,
                  'sig_baseline': 10.0,
                  'prctile_baseline': 8.0,
                  'neucoeff': 0.7,
                  'xrange': np.array([0, 0]),
                  'yrange': np.array([0, 0])}


In [ ]:
imaging.ProcessingParamSet.insert_new_params(
    processing_method="suite2p",
    concatenation_method="concat",
    paramset_idx=1,
    params=params_suite2p,
    paramset_desc="TR: Mini2p (rigid, built-in classifier, scan_concat)",
)


In [ ]:
# Insert the param_set - Trondheim Mini2p

# TODO: Parameter set needs to be updated with ScanInfo settings

params_suite2p = {'look_one_level_down': False,
                  'fast_disk': dj.config['custom'].get('suite2p_fast_tmp')[0],
                  'delete_bin': True,
                  'mesoscan': False,
                  'h5py': [],
                  'h5py_key': 'data',
                  'save_path0': [],
                  'subfolders': [],
                  'nplanes': 1,
                  'nchannels': 1,
                  'functional_chan': 1,
                  'tau': 1.0,
                  'fs': 15.3845,
                  'force_sktiff': False,
                  'preclassify': 0.0,
                  'save_mat': True,
                  'combined': True,
                  'aspect': 1.0,
                  'do_bidiphase': True,
                  'bidiphase': 0.0,
                  'do_registration': True,
                  'two_step_registration': True,
                  'norm_frames': True,
                  'keep_movie_raw': False,
                  'nimg_init': 1000,
                  'batch_size': 8000,
                  'maxregshift': 0.1,
                  'align_by_chan': 1,
                  'reg_tif': False,
                  'reg_tif_chan2': False,
                  'subpixel': 10,
                  'smooth_sigma_time': 1, 
                  'smooth_sigma': 1.15,
                  'th_badframes': 1.0,
                  'pad_fft': False,
                  'nonrigid': True,
                  'block_size': [64, 64],
                  'snr_thresh': 1.2,
                  'maxregshiftNR': 5.0,
                  '1Preg': False,
                  'spatial_hp': 50.0,
                  'pre_smooth': 2.0,
                  'spatial_taper': 50.0,
                  'roidetect': True,
                  'sparse_mode': True,
                  'denoise': True,
                  'diameter': 6,
                  'spatial_scale': 0,
                  'connected': True,
                  'nbinned': 9000,
                  'max_iterations': 30,
                  'threshold_scaling': 1.0,
                  'max_overlap': 0.75,
                  'high_pass': 100.0,
                  'inner_neuropil_radius': 2,
                  'min_neuropil_pixels': 350,
                  'allow_overlap': False,
                  'chan2_thres': 0.65,
                  'baseline': 'maximin',
                  'win_baseline': 60.0,
                  'sig_baseline': 10.0,
                  'prctile_baseline': 8.0,
                  'neucoeff': 0.7,
                  'xrange': np.array([0, 0]),
                  'yrange': np.array([0, 0])}


In [ ]:
imaging.ProcessingParamSet.insert_new_params(
    processing_method="suite2p",
    concatenation_method="concat",
    paramset_idx=2,
    params=params_suite2p,
    paramset_desc="TR: Trondheim Mini2p (non-rigid, built-in classifier, scan_concat)",
)


In [ ]:
# Insert the param_set - Bench2p

# example for setting a link for the classifier to be used for cell curation. Needs
classifier_folder = "/home/tobiasr/adamacs/user_data/s2p_classifiers/TR_HPC_bench2p_classifier.npy"


params_suite2p = {'look_one_level_down': False,
                  'fast_disk': dj.config['custom'].get('suite2p_fast_tmp')[0],
                  'delete_bin': True,
                  'mesoscan': False,
                  'h5py': [],
                  'h5py_key': 'data',
                  'save_path0': [],
                  'subfolders': [],
                  'nplanes': 1,
                  'nchannels': 1,
                  'functional_chan': 1,
                  'tau': 1.0,
                  'fs': 29.9784,
                  'force_sktiff': False,
                  'preclassify': 0.0,
                  'use_builtin_classifier': False,
                  'classifier_path': classifier_folder,
                  'save_mat': True,
                  'combined': True,
                  'aspect': 1.0,
                  'do_bidiphase': True,
                  'bidiphase': 0.0,
                  'do_registration': True,
                  'two_step_registration': False,
                  'norm_frames': True,
                  'keep_movie_raw': False,
                  'nimg_init': 1000,
                  'batch_size': 8000,
                  'maxregshift': 0.1,
                  'align_by_chan': 1,
                  'reg_tif': False,
                  'reg_tif_chan2': False,
                  'subpixel': 10,
                  'smooth_sigma_time': 1, 
                  'smooth_sigma': 1.15,
                  'th_badframes': 1.0,
                  'pad_fft': False,
                  'nonrigid': True,
                  'block_size': [128, 128],
                  'snr_thresh': 1.2,
                  'maxregshiftNR': 5.0,
                  '1Preg': False,
                  'spatial_hp': 50.0,
                  'pre_smooth': 2.0,
                  'spatial_taper': 50.0,
                  'roidetect': True,
                  'sparse_mode': True,
                  'denoise': True,
                  'diameter': 12,
                  'spatial_scale': 0,
                  'connected': True,
                  'nbinned': 9000,
                  'max_iterations': 30,
                  'threshold_scaling': 1.0,
                  'max_overlap': 0.75,
                  'high_pass': 100.0,
                  'inner_neuropil_radius': 2,
                  'min_neuropil_pixels': 350,
                  'allow_overlap': False,
                  'chan2_thres': 0.65,
                  'baseline': 'maximin',
                  'win_baseline': 60.0,
                  'sig_baseline': 10.0,
                  'prctile_baseline': 8.0,
                  'neucoeff': 0.7,
                  'xrange': np.array([0, 0]),
                  'yrange': np.array([0, 0])}


In [ ]:
imaging.ProcessingParamSet.insert_new_params(
    processing_method="suite2p",
    concatenation_method="concat",
    paramset_idx=3,
    params=params_suite2p,
    paramset_desc="TR: Bench2p (non-rigid, custom classifier, scan_concat)",
)


In [ ]:
# Insert the param_set - RIGID Bench2p

classifier_folder = "/home/tobiasr/adamacs/user_data/s2p_classifiers/TR_HPC_bench2p_classifier.npy"

params_suite2p = {'look_one_level_down': False,
                  'fast_disk': dj.config['custom'].get('suite2p_fast_tmp')[0],
                  'delete_bin': True,
                  'mesoscan': False,
                  'h5py': [],
                  'h5py_key': 'data',
                  'save_path0': [],
                  'subfolders': [],
                  'nplanes': 1,
                  'nchannels': 1,
                  'functional_chan': 1,
                  'tau': 1.0,
                  'fs': 29.9784,
                  'force_sktiff': False,
                  'preclassify': 0.0,
                  'use_builtin_classifier': False,
                  'classifier_path': classifier_folder,
                  'save_mat': True,
                  'combined': True,
                  'aspect': 1.0,
                  'do_bidiphase': True,
                  'bidiphase': 0.0,
                  'do_registration': True,
                  'two_step_registration': False,
                  'norm_frames': True,
                  'keep_movie_raw': False,
                  'nimg_init': 1000,
                  'batch_size': 8000,
                  'maxregshift': 0.1,
                  'align_by_chan': 1,
                  'reg_tif': False,
                  'reg_tif_chan2': False,
                  'subpixel': 10,
                  'smooth_sigma_time': 1, 
                  'smooth_sigma': 1.15,
                  'th_badframes': 1.0,
                  'pad_fft': False,
                  'nonrigid': False,
                  'block_size': [128, 128],
                  'snr_thresh': 1.2,
                  'maxregshiftNR': 5.0,
                  '1Preg': False,
                  'spatial_hp': 50.0,
                  'pre_smooth': 2.0,
                  'spatial_taper': 50.0,
                  'roidetect': True,
                  'sparse_mode': True,
                  'denoise': True,
                  'diameter': 12,
                  'spatial_scale': 0,
                  'connected': True,
                  'nbinned': 9000,
                  'max_iterations': 30,
                  'threshold_scaling': 1.0,
                  'max_overlap': 0.75,
                  'high_pass': 100.0,
                  'inner_neuropil_radius': 2,
                  'min_neuropil_pixels': 350,
                  'allow_overlap': False,
                  'chan2_thres': 0.65,
                  'baseline': 'maximin',
                  'win_baseline': 60.0,
                  'sig_baseline': 10.0,
                  'prctile_baseline': 8.0,
                  'neucoeff': 0.7,
                  'xrange': np.array([0, 0]),
                  'yrange': np.array([0, 0])}


In [ ]:
# (imaging.ProcessingParamSet & "paramset_idx = 4").delete()
imaging.ProcessingParamSet.insert_new_params(
    processing_method="suite2p",
    concatenation_method="concat",
    paramset_idx=4,
    params=params_suite2p,
    paramset_desc="TR: Bench2p (rigid, custom classifier, scan_concat)",
)


In [ ]:
imaging.ProcessingParamSet()

In [ ]:
# Insert the param_set - Mini2p - rigid

# example for setting a link fro the classifier to be used for cell curation. Needs
classifier_folder = "/home/tobiasr/adamacs/user_data/s2p_classifiers/TR_HPC_bench2p_classifier.npy"

params_suite2p = {'look_one_level_down': False,
                  'fast_disk': dj.config['custom'].get('suite2p_fast_tmp')[0],
                  'delete_bin': True,
                  'mesoscan': False,
                  'h5py': [],
                  'h5py_key': 'data',
                  'save_path0': [],
                  'subfolders': [],
                  'nplanes': 1,
                  'nchannels': 1,
                  'functional_chan': 1,
                  'tau': 1.0,
                  'fs': 15.3845,
                  'force_sktiff': False,
                  'preclassify': 0.0,
                  'use_builtin_classifier': True,
                #   'classifier_path': classifier_folder,
                  'save_mat': True,
                  'combined': True,
                  'aspect': 1.0,
                  'do_bidiphase': True,
                  'bidiphase': 0.0,
                  'do_registration': True,
                  'two_step_registration': True,
                  'norm_frames': True,
                  'keep_movie_raw': False,
                  'nimg_init': 1000,
                  'batch_size': 8000,
                  'maxregshift': 0.1,
                  'align_by_chan': 1,
                  'reg_tif': False,
                  'reg_tif_chan2': False,
                  'subpixel': 10,
                  'smooth_sigma_time': 1, 
                  'smooth_sigma': 1.15,
                  'th_badframes': 1.0,
                  'pad_fft': False,
                  'nonrigid': False,
                  'block_size': [64, 64],
                  'snr_thresh': 1.2,
                  'maxregshiftNR': 5.0,
                  '1Preg': False,
                  'spatial_hp': 50.0,
                  'pre_smooth': 2.0,
                  'spatial_taper': 50.0,
                  'roidetect': True,
                  'sparse_mode': True,
                  'denoise': True,
                  'diameter': 6,
                  'spatial_scale': 0,
                  'connected': True,
                  'nbinned': 9000,
                  'max_iterations': 30,
                  'threshold_scaling': 1.0,
                  'max_overlap': 0.75,
                  'high_pass': 100.0,
                  'inner_neuropil_radius': 2,
                  'min_neuropil_pixels': 350,
                  'allow_overlap': False,
                  'chan2_thres': 0.65,
                  'baseline': 'maximin',
                  'win_baseline': 50.0,
                  'sig_baseline': 10.0,
                  'prctile_baseline': 8.0,
                  'neucoeff': 0.7,
                  'xrange': np.array([0, 0]),
                  'yrange': np.array([0, 0])}


In [ ]:
imaging.ProcessingParamSet.insert_new_params(
    processing_method="suite2p",
    concatenation_method="indiv",
    paramset_idx=5,
    params=params_suite2p,
    paramset_desc="TR: Mini2p (rigid, built-in classifier, individual)",
)


In [ ]:
# Insert the param_set - Mini2p - rigid - crosssession concatenation

# example for setting a link fro the classifier to be used for cell curation. Needs
classifier_folder = "/home/tobiasr/adamacs/user_data/s2p_classifiers/TR_HPC_bench2p_classifier.npy"

params_suite2p = {'look_one_level_down': False,
                  'fast_disk': dj.config['custom'].get('suite2p_fast_tmp')[0],
                  'delete_bin': True,
                  'mesoscan': False,
                  'h5py': [],
                  'h5py_key': 'data',
                  'save_path0': [],
                  'subfolders': [],
                  'nplanes': 1,
                  'nchannels': 1,
                  'functional_chan': 1,
                  'tau': 1.0,
                  'fs': 15.3845,
                  'force_sktiff': False,
                  'preclassify': 0.0,
                  'use_builtin_classifier': True,
                #   'classifier_path': classifier_folder,
                  'save_mat': True,
                  'combined': True,
                  'aspect': 1.0,
                  'do_bidiphase': True,
                  'bidiphase': 0.0,
                  'do_registration': True,
                  'two_step_registration': True,
                  'norm_frames': True,
                  'keep_movie_raw': False,
                  'nimg_init': 1000,
                  'batch_size': 8000,
                  'maxregshift': 0.1,
                  'align_by_chan': 1,
                  'reg_tif': False,
                  'reg_tif_chan2': False,
                  'subpixel': 10,
                  'smooth_sigma_time': 1, 
                  'smooth_sigma': 1.15,
                  'th_badframes': 1.0,
                  'pad_fft': False,
                  'nonrigid': False,
                  'block_size': [64, 64],
                  'snr_thresh': 1.2,
                  'maxregshiftNR': 5.0,
                  '1Preg': False,
                  'spatial_hp': 50.0,
                  'pre_smooth': 2.0,
                  'spatial_taper': 50.0,
                  'roidetect': True,
                  'sparse_mode': True,
                  'denoise': True,
                  'diameter': 6,
                  'spatial_scale': 0,
                  'connected': True,
                  'nbinned': 9000,
                  'max_iterations': 30,
                  'threshold_scaling': 1.0,
                  'max_overlap': 0.75,
                  'high_pass': 100.0,
                  'inner_neuropil_radius': 2,
                  'min_neuropil_pixels': 350,
                  'allow_overlap': False,
                  'chan2_thres': 0.65,
                  'baseline': 'maximin',
                  'win_baseline': 60.0,
                  'sig_baseline': 10.0,
                  'prctile_baseline': 8.0,
                  'neucoeff': 0.7,
                  'xrange': np.array([0, 0]),
                  'yrange': np.array([0, 0])}


In [ ]:
imaging.ProcessingParamSet.insert_new_params(
    processing_method="suite2p",
    concatenation_method="consame",
    paramset_idx=6,
    params=params_suite2p,
    paramset_desc="TR: Mini2p (rigid, built-in classifier, SAMESITE_concat)",
)


In [ ]:
imaging.ProcessingParamSet()

In [ ]:
imaging.Processing.populate(display_progress=True,  suppress_errors=False)